# Семинар 7: Усложнение модели, борьба с переобучением и надежная оценка

**Цели семинара:**
1.  Реализовать полиномиальную регрессию для моделирования нелинейных зависимостей.
2.  Научиться диагностировать переобучение с помощью кривых обучения.
3.  Применить кросс-валидацию (`cross_val_score`) для получения робастной оценки качества модели.
4.  Изучить и применить модели с регуляризацией: `Ridge`, `Lasso`, `ElasticNet`.
5.  Использовать `GridSearchCV` для автоматического подбора оптимального коэффициента регуляризации (`alpha`).

## 1. Подготовка: загружаем данные и обучаем базовую модель

Мы продолжим работать с данными `Advertising.csv` и будем отталкиваться от простой линейной модели, которую мы построили на прошлом семинаре.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Загружаем данные
df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/Advertising.csv')

# Готовим X и y
X = df.drop('Sales', axis=1)
y = df['Sales']

# Важно! Для дальнейшей работы нам понадобятся только обучающие и тестовые данные
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 2. Полиномиальная регрессия

Наша базовая линейная модель была неплохой, но возможно, зависимости в данных более сложные. Попробуем усложнить модель, добавив полиномиальные признаки (например, $TV^2$, $Radio^2$, а также их взаимодействия $TV \times Radio$).

Для этого мы создадим конвейер (`pipeline`), который будет последовательно выполнять два шага:
1.  `PolynomialFeatures()`: Генерировать новые признаки.
2.  `LinearRegression()`: Обучать линейную регрессию на этих новых, сгенерированных признаках.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

# Создаем конвейер для полиномиальной регрессии 2-й степени
poly_model = make_pipeline(PolynomialFeatures(degree=2, include_bias=False),
                           LinearRegression())

# Обучаем модель
poly_model.fit(X_train, y_train)

### 2.1. Оценка полиномиальной модели

Сравним качество новой, более сложной модели, с нашей старой простой моделью.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Предсказания полиномиальной модели
y_poly_pred = poly_model.predict(X_test)

# Метрики
mae_poly = mean_absolute_error(y_test, y_poly_pred)
rmse_poly = np.sqrt(mean_squared_error(y_test, y_poly_pred))

print(f"Полиномиальная модель (2 степень):")
print(f"MAE: {mae_poly:.2f}") # Было 1.51 у простой модели
print(f"RMSE: {rmse_poly:.2f}") # Было 1.93 у простой модели

**Вывод:** Усложнение модели сработало! Ошибки MAE и RMSE значительно уменьшились. Это говорит о том, что в данных действительно были нелинейные зависимости (вероятно, взаимодействия признаков), которые новая модель смогла уловить.

### 2.2. Поиск оптимальной сложности (степени полинома)

Мы выбрали степень `degree=2` интуитивно. А вдруг `degree=3` или `degree=5` еще лучше? Или, может, мы уже переобучились? Построим **кривые обучения**, чтобы найти оптимальную степень.

In [ ]:
train_rmse_errors = []
test_rmse_errors = []
degrees = range(1, 10)

for d in degrees:
    # Создаем и обучаем модель для каждой степени
    poly_converter = PolynomialFeatures(degree=d, include_bias=False)
    X_poly_train = poly_converter.fit_transform(X_train)
    X_poly_test = poly_converter.transform(X_test) # Важно: используем transform, а не fit_transform
    
    model = LinearRegression()
    model.fit(X_poly_train, y_train)
    
    # Делаем предсказания и считаем ошибки
    y_train_pred = model.predict(X_poly_train)
    y_test_pred = model.predict(X_poly_test)
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    train_rmse_errors.append(train_rmse)
    test_rmse_errors.append(test_rmse)

# Визуализируем кривые обучения
plt.figure(figsize=(10, 6))
plt.plot(degrees, train_rmse_errors, label='Train RMSE')
plt.plot(degrees, test_rmse_errors, label='Test RMSE')
plt.xlabel("Степень полинома")
plt.ylabel("RMSE")
plt.legend()
plt.grid()
plt.show()

**Вывод:** График четко показывает, что ошибка на тестовой выборке (синяя линия) минимальна при степени 2-3. Дальнейшее усложнение модели (степень 4 и выше) приводит к росту тестовой ошибки, хотя ошибка на обучении продолжает падать. Это и есть **переобучение**. Значит, мы были правы, выбрав степень 2 или 3.

## 3. Регуляризация: Борьба с переобучением

Теперь представим, что у нас очень много признаков, и полиномиальная модель высокой степени все равно показывает лучшие результаты на валидации. Чтобы сделать ее более стабильной и предотвратить переобучение, мы можем применить регуляризацию.

**Важно:** Для регуляризации **обязательно** нужно масштабировать данные! Мы добавим `StandardScaler` в наш конвейер.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet

# Создадим пайплайн с тремя шагами: масштабирование, генерация признаков, модель
# Сразу возьмем степень полинома = 3, как одну из оптимальных

# Ridge (L2)
ridge_pipe = make_pipeline(StandardScaler(),
                           PolynomialFeatures(degree=3, include_bias=False),
                           Ridge(alpha=1.0))
ridge_pipe.fit(X_train, y_train)
y_ridge_pred = ridge_pipe.predict(X_test)
print(f"RMSE для Ridge: {np.sqrt(mean_squared_error(y_test, y_ridge_pred)):.2f}")

# Lasso (L1)
lasso_pipe = make_pipeline(StandardScaler(),
                           PolynomialFeatures(degree=3, include_bias=False),
                           Lasso(alpha=0.01))
lasso_pipe.fit(X_train, y_train)
y_lasso_pred = lasso_pipe.predict(X_test)
print(f"RMSE для Lasso: {np.sqrt(mean_squared_error(y_test, y_lasso_pred)):.2f}")

### 3.1. Подбор оптимального `alpha` с помощью кросс-валидации

Мы выбрали `alpha` наугад. Правильный способ — найти оптимальное значение с помощью поиска по сетке с кросс-валидацией (`GridSearchCV`).

In [ ]:
from sklearn.model_selection import GridSearchCV

# Пайплайн для ElasticNet (комбинация L1 и L2)
elastic_pipe = make_pipeline(StandardScaler(),
                             PolynomialFeatures(degree=3, include_bias=False),
                             ElasticNet(max_iter=10000))

# Сетка гиперпараметров, которые мы хотим проверить
param_grid = {
    'elasticnet__alpha': [0.001, 0.01, 0.1, 1, 10, 100],
    'elasticnet__l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
}

# Создаем объект GridSearchCV. cv=5 означает 5-fold кросс-валидацию.
# scoring='neg_mean_squared_error' - метрика для оптимизации.
grid_search = GridSearchCV(estimator=elastic_pipe, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='neg_mean_squared_error',
                           verbose=0)

# Запускаем поиск
grid_search.fit(X_train, y_train)

### 3.2. Результаты поиска

Посмотрим, какие гиперпараметры оказались лучшими.

In [ ]:
print("Лучшие параметры:", grid_search.best_params_)

# Оценим лучшую найденную модель на тестовой выборке
best_model = grid_search.best_estimator_
y_best_pred = best_model.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_best_pred))
print(f"\nФинальный RMSE на тестовой выборке: {rmse_best:.2f}")

### 3.3. Визуализация эффекта регуляризации

Мы увидели, как регуляризация влияет на коэффициенты. А как это выглядит на самом графике? Давайте обучим три полиномиальные модели высокой степени (например, 15) на наших синтетических данных: одну без регуляризации, одну с Ridge и одну с Lasso. Это позволит наглядно увидеть "сглаживающий" эффект.

In [ ]:
# Используем данные, сгенерированные в начале лекции
# Для полноты примера, определим их здесь снова.
np.random.seed(0)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)

# Данные для построения гладких кривых
X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)

# Создаем пайплайны для трех моделей. Обратите внимание на StandardScaler!
degree = 15
pipe_lr = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree, include_bias=False), LinearRegression())
pipe_ridge = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree, include_bias=False), Ridge(alpha=1))
pipe_lasso = make_pipeline(StandardScaler(), PolynomialFeatures(degree=degree, include_bias=False), Lasso(alpha=0.1, max_iter=100000))

# Обучаем модели
pipe_lr.fit(X_poly_data, y_poly_data)
pipe_ridge.fit(X_poly_data, y_poly_data)
pipe_lasso.fit(X_poly_data, y_poly_data)

# Строим графики
plt.figure(figsize=(12, 8))
plt.scatter(X_poly_data, y_poly_data, alpha=0.4, label='Исходные данные')

# График для обычной регрессии
y_plot_lr = pipe_lr.predict(X_plot)
plt.plot(X_plot, y_plot_lr, label='Линейная регрессия (переобучение)', color='red', linestyle='--')

# График для Ridge
y_plot_ridge = pipe_ridge.predict(X_plot)
plt.plot(X_plot, y_plot_ridge, label='Ridge (L2) регрессия', color='green', linewidth=3)

# График для Lasso
y_plot_lasso = pipe_lasso.predict(X_plot)
plt.plot(X_plot, y_plot_lasso, label='Lasso (L1) регрессия', color='purple', linewidth=3)

plt.legend()
plt.ylim(0, 10)
plt.title('Сравнение моделей с регуляризацией и без нее')
plt.grid(True)
plt.show()

**Вывод из графика:**
*   **Красная пунктирная линия (Линейная регрессия):** Ведет себя нестабильно, сильно изгибается, пытаясь "поймать" каждую точку. Это яркий пример переобучения.
*   **Зеленая линия (Ridge):** Гораздо более гладкая и лучше отражает общую квадратичную тенденцию в данных, игнорируя локальные выбросы. Она "усмирила" модель.
*   **Фиолетовая линия (Lasso):** Также является гладкой и робастной. В данном случае она очень похожа на Ridge, но в других ситуациях, где много неинформативных признаков, она могла бы дать еще более простую модель (ближе к параболе).

Этот график наглядно демонстрирует, как регуляризация, штрафуя за большие коэффициенты, заставляет модель находить более простые и более гладкие решения, что улучшает ее обобщающую способность.

## Заключение

На этом семинаре мы сделали большой шаг вперед:
*   Научились моделировать нелинейности с помощью **полиномиальной регрессии**.
*   Увидели проблему **переобучения** на кривых обучения и поняли, как выбирать оптимальную сложность модели.
*   Применили **регуляризацию** (Ridge, Lasso, ElasticNet) для борьбы с переобучением и отбора признаков.
*   Использовали **GridSearchCV** — мощный инструмент для автоматического подбора гиперпараметров с помощью кросс-валидации.

В результате мы смогли построить сложную, но при этом робастную модель и получили еще более низкую ошибку на тестовых данных, чем раньше.